To perform statistical entropy analysis in Python for EEG data, you can use libraries such as nolds or pyEEG. These libraries provide functions to calculate different types of entropy measures, including approximate entropy (ApEn), sample entropy (SampEn), and permutation entropy (PermEn)

Shannon Entropy: Shannon entropy is a widely used measure of uncertainty or information content in a signal. It quantifies the amount of information or randomness present in the signal. It can be useful for analyzing the overall complexity of EEG signals.

Approximate Entropy (ApEn): Approximate entropy is a measure of regularity or complexity in time series data. It quantifies the likelihood that similar patterns repeat in the signal. ApEn can be helpful in assessing the complexity of EEG signals and detecting changes in regularity.

Sample Entropy (SampEn): Sample entropy is a variation of approximate entropy that addresses some limitations of ApEn. It measures the likelihood of similar patterns repeating in a signal while also taking into account self-matches of shorter patterns. SampEn can provide insights into the complexity and irregularity of EEG signals.

Renyi Entropy: Renyi entropy is a generalization of Shannon entropy that includes a parameter to control the level of sensitivity to different probability distributions. It provides a measure of the diversity or variability in the signal.

Permutation Entropy: Permutation entropy measures the amount of disorder or randomness in the order of patterns in a signal. It considers the relative order of values rather than their actual magnitudes. Permutation entropy can be useful in characterizing the complexity and irregularity of EEG signals.

-----------
Permutation Entropy
----------

In [29]:
import numpy as np
from scipy.stats import entropy
import mne, os
from mne import Epochs
from pyentrp import entropy as py_entropy

mne.set_log_level("ERROR")
raw_folder = os.path.join(os.path.expanduser('~/'),'Desktop', 'FYP', 'code_env', 'eeg-notebooks','FYP', 'data_ordered', 'mne_raw')
raw_files = [file for file in os.listdir(raw_folder) if file.endswith(".fif")]
marker_mapping = {"blue": 1, "red": 2, "right": 3, "left": 4, "right arrow": 5, "left arrow": 6}
duration = 30.0  # Duration of each epoch (seconds)
event_ids = {'left': 4, 'right': 3}  # Replace with your event IDs
tmin, tmax = -0.3, 0.7
evokeds_left = {}
evokeds_right = {}

entropy_audio_dic = {"TP9":[], "AF7":[], "AF8":[], "TP10":[]}
entropy_shape_dic = {"TP9":[], "AF7":[], "AF8":[], "TP10":[]}
entropy_vibro_dic = {"TP9":[], "AF7":[], "AF8":[], "TP10":[]}

# Process each raw file and extract evoked response
for file in raw_files:
    print("Computing entropy for trial",file)
    raw_path = os.path.join(raw_folder, file)
    raw = mne.io.read_raw_fif(raw_path, preload=True)
    data = raw.get_data()
    events, event_id= mne.events_from_annotations(raw, event_id=marker_mapping)
    epochs = Epochs(raw, events=events, event_id=event_ids, tmin=-0.3, tmax=0.7, baseline=None, preload=True)

    # Compute entropy for each channel
    entropy_values = []
    for ch_name in raw.ch_names:
        # This will give you an Epochs object containing only the data for the current channel
        channel_epochs = epochs.copy().pick_channels([ch_name])
        channel_data = channel_epochs.get_data().reshape(-1)
        channel_entropy = py_entropy.permutation_entropy(channel_data, 3, 1)
        entropy_values.append(channel_entropy)
        
        if not np.any(np.isnan(entropy_values)):
            if "Shape" in file:
                entropy_shape_dic[ch_name].append(entropy_values)   
            elif "Audio" in file:
                entropy_audio_dic[ch_name].append(entropy_values) 
            elif "Vibro" in file:
                entropy_vibro_dic[ch_name].append(entropy_values)

    # # Print the entropy values for each channel
    # for channel_idx, entropy_value in enumerate(entropy_values):
    #     print(f'Channel {channel_idx + 1}: {entropy_value}')
            



Computing entropy for trial AudioVisual_01_1.fif
Computing entropy for trial AudioVisual_01_2.fif
Computing entropy for trial AudioVisual_02_1.fif
Computing entropy for trial AudioVisual_03_1.fif
Computing entropy for trial AudioVisual_03_2.fif
Computing entropy for trial AudioVisual_04_1.fif
Computing entropy for trial AudioVisual_04_2.fif
Computing entropy for trial AudioVisual_05_1.fif
Computing entropy for trial AudioVisual_05_2.fif
Computing entropy for trial AudioVisual_06_1.fif
Computing entropy for trial AudioVisual_06_2.fif
Computing entropy for trial AudioVisual_07_1.fif
Computing entropy for trial AudioVisual_07_2.fif
Computing entropy for trial AudioVisual_09_1.fif
Computing entropy for trial AudioVisual_09_2.fif
Computing entropy for trial AudioVisual_10_1.fif
Computing entropy for trial AudioVisual_10_2.fif
Computing entropy for trial AudioVisual_11_1.fif
Computing entropy for trial AudioVisual_11_2.fif
Computing entropy for trial ShapeVisual_01_1.fif
Computing entropy fo

In [31]:
print(entropy_audio_dic['TP9'])
print(entropy_audio_dic['TP10'])

[[2.0612664108109637, 2.0292518574112974, 2.058857793322028, 2.0731944722077813], [2.0346540913198776, 2.064217333318041, 2.0576213799920864, 2.0769552827648967], [2.0814987090868904, 2.0758498985884826, 2.078096720894668, 2.071980608304488], [1.9631552000694288, 1.990835164554389, 2.0026754289728728, 1.9666489466731796], [1.9829858733176153, 2.009669303694042, 2.0137756031819136, 2.0022665103664665], [1.9068689648501156, 2.0130001495357184, 2.0069722586904173, 1.979931807205778], [1.8915160894822913, 2.0068568248522514, 1.9950411385929419, 1.9276964753204173], [1.9954270305428938, 2.07621938007862, 2.031033505048538, 1.9330824517891063], [1.9724640849553605, 2.0741258941036045, 2.0264447491261923, 2.0355970849929004], [1.831794434488342, 2.0089593700442077, 1.9509863852075813, 1.8680582357539113], [1.8512668791209375, 2.0033209978398663, 1.9990335276741753, 1.9228649659046615], [2.079842882240269, 2.0837170952498485, 2.0946435109109305, 2.0744067301426927], [2.0773609759201865, 2.0856

In [47]:
import matplotlib.pyplot as plt
import numpy as np
import json
ch= 'TP9'
entropy_audio = np.array(entropy_audio_dic[ch])
audio_entropy_mean = np.mean(entropy_audio, axis=0)
audio_entropy_std = np.std(entropy_audio, axis=0)
print(entropy_audio.shape)
entropy_vibro = np.array(entropy_vibro_dic[ch])
vibro_entropy_mean = np.mean(entropy_vibro, axis=0)
vibro_entropy_std = np.std(entropy_vibro, axis=0)

entropy_shape = np.array(entropy_shape_dic[ch])
shape_entropy_mean = np.mean(entropy_shape, axis=0)
shape_entropy_std = np.std(entropy_shape, axis=0)

# entropy_audio = entropy_audio[np.isfinite(entropy_audio)]
# entropy_vibro = entropy_vibro[np.isfinite(entropy_vibro)]
# entropy_shape = entropy_shape[np.isfinite(entropy_shape)]

json_file_path = 'entropy.json'
entropy = {"Audio": entropy_audio, "Vibro": entropy_vibro, "Shape": entropy_shape}

print("Audio", audio_entropy_mean, audio_entropy_std)
print("Vibro", vibro_entropy_mean, vibro_entropy_std)
print("Shape", shape_entropy_mean, shape_entropy_std)

#num_channels = entropy_audio.shape[1]

# Generate x-axis values (assuming equal length arrays)
x = np.arange(28)

# # Plotting for each channel
# #for channel in range(1):
# channel = 0
# plt.plot(x, entropy['Audio'][0,:28], label=f'Audio Channel TP9', marker='o')
# plt.plot(x, entropy['Vibro'][0,:28], label=f'Vibro Channel TP9', marker='o')
# plt.plot(x, entropy['Shape'][0,:28], label=f'Shape Channel TP9', marker='o')

# # Add labels, title, and legend for each plot
# plt.xlabel('Epoch Number')
# plt.ylabel('Entropy')
# plt.title(f'Entropy Comparison for Channel {ch}')
# plt.legend()

# # Show the plot
# plt.show()

entropy_array = (np.array(entropy.copy).shape)
# Save the variables to a JSON file
with open(json_file_path, 'w') as file:
    json.dump(entropy_array, file, indent=4)
print("Variables saved to JSON file:", json_file_path)

(19, 4)
Audio [1.98979372 2.02908398 2.02120699 2.0015689 ] [0.08395778 0.04355176 0.05594295 0.07315489]
Vibro [1.99636447 2.0117672  2.01831004 1.99697123] [0.09564352 0.10270661 0.08100715 0.09984612]
Shape [1.96436353 2.00619902 1.9961825  1.96750563] [0.13272192 0.11402008 0.11391364 0.12833112]
Variables saved to JSON file: entropy.json


-------
Statistical Analysis SE
--------

In [48]:
print(entropy['Audio'])

[[2.06126641 2.02925186 2.05885779 2.07319447]
 [2.03465409 2.06421733 2.05762138 2.07695528]
 [2.08149871 2.0758499  2.07809672 2.07198061]
 [1.9631552  1.99083516 2.00267543 1.96664895]
 [1.98298587 2.0096693  2.0137756  2.00226651]
 [1.90686896 2.01300015 2.00697226 1.97993181]
 [1.89151609 2.00685682 1.99504114 1.92769648]
 [1.99542703 2.07621938 2.03103351 1.93308245]
 [1.97246408 2.07412589 2.02644475 2.03559708]
 [1.83179443 2.00895937 1.95098639 1.86805824]
 [1.85126688 2.003321   1.99903353 1.92286497]
 [2.07984288 2.0837171  2.09464351 2.07440673]
 [2.07736098 2.08560328 2.09400519 2.0744274 ]
 [1.9513066  1.93831271 1.8826294  2.02246477]
 [1.94971357 1.93235024 1.91298722 1.93520164]
 [1.9814004  2.05205804 2.02621433 1.9479453 ]
 [1.94641818 2.04680874 2.03975675 1.90674561]
 [2.12097339 2.03710955 2.04810389 2.11253989]
 [2.12616693 2.02432979 2.08405396 2.09780095]]


In [53]:
import scipy.stats as stats
from scipy.stats import f_oneway
# Perform the Brown-Forsythe test
num_channels = 4#entropy_audio.shape[1]

# Generate x-axis values (assuming equal length arrays)
x = np.arange(14)

print("A-V")
# Plotting for each channel
for channel in range(num_channels):
    _, p_value = stats.levene(entropy['Audio'][:, channel], entropy['Vibro'][:, channel], center='median')

    # Print the results
    if p_value < 0.05:
        print("The variances are significantly different (p < 0.05), violating the assumption of homogeneous variance.")
    else:
        print("The variances are not significantly different (p >= 0.05), supporting the assumption of homogeneous variance.", p_value)

    f_value, p_value = f_oneway(entropy['Audio'][:, channel], entropy['Vibro'][:, channel])

    print(f"ANOVA results for channel {channel}")
    print("p-values:", p_value )

print("A-S")
# Plotting for each channel
for channel in range(num_channels):
    _, p_value = stats.levene(entropy['Audio'][:, channel], entropy['Shape'][:, channel], center='median')

    # Print the results
    if p_value < 0.05:
        print("The variances are significantly different (p < 0.05), violating the assumption of homogeneous variance.")
    else:
        print("The variances are not significantly different (p >= 0.05), supporting the assumption of homogeneous variance.", p_value)

    f_value, p_value = f_oneway(entropy['Audio'][:, channel], entropy['Shape'][:, channel])

    print(f"ANOVA results for {channel}")
    print("p-values:", p_value )

print("S-V")
# Plotting for each channel
for channel in range(num_channels):
    _, p_value = stats.levene(entropy['Shape'][:, channel], entropy['Vibro'][:, channel], center='median')

    # Print the results
    if p_value < 0.05:
        print("The variances are significantly different (p < 0.05), violating the assumption of homogeneous variance.")
    else:
        print("The variances are not significantly different (p >= 0.05), supporting the assumption of homogeneous variance.", p_value)

    f_value, p_value = f_oneway(entropy['Shape'][:, channel], entropy['Vibro'][:, channel])

    print(f"ANOVA results for {channel}")
    print("p-values:", p_value )


A-V
The variances are not significantly different (p >= 0.05), supporting the assumption of homogeneous variance. 0.698541671880511
ANOVA results for channel 0
p-values: 0.8258217631104077
The variances are not significantly different (p >= 0.05), supporting the assumption of homogeneous variance. 0.3431983093388873
ANOVA results for channel 1
p-values: 0.512352574626793
The variances are not significantly different (p >= 0.05), supporting the assumption of homogeneous variance. 0.5757909625468969
ANOVA results for channel 2
p-values: 0.9004488264459951
The variances are not significantly different (p >= 0.05), supporting the assumption of homogeneous variance. 0.4240888292551711
ANOVA results for channel 3
p-values: 0.8744562336609205
A-S
The variances are not significantly different (p >= 0.05), supporting the assumption of homogeneous variance. 0.3422451034098387
ANOVA results for 0
p-values: 0.5044375808679135
The variances are not significantly different (p >= 0.05), supporting th

-------------
Approximate Entropy ApEn
--------------

In [1]:
#------------approximate Entropy ApEn

import numpy as np
import pyeeg

# Load the EEG data as a NumPy array
data = np.loadtxt('eeg_data.txt')

# Compute approximate entropy for each channel
entropy_values = []
for channel_data in data:
    entropy_value = pyeeg.ap_entropy(channel_data)
    entropy_values.append(entropy_value)

# Print the entropy values for each channel
for channel_idx, entropy_value in enumerate(entropy_values):
    print(f'Channel {channel_idx + 1}: {entropy_value}')


ModuleNotFoundError: No module named 'pyeeg'

In [ ]:
import numpy as np
from nolds import entropy
import mne

# Load the EEG data using MNE Raw object
raw = mne.io.read_raw_edf('your_eeg_file.edf', preload=True)

# Extract the data as a NumPy array
data = raw.get_data()

# Compute statistical entropy for each channel
entropy_values = []
for channel_data in data:
    entropy_value = entropy(channel_data, emb_dim=10, r=0.1)
    entropy_values.append(entropy_value)

# Print the entropy values for each channel
for channel_idx, entropy_value in enumerate(entropy_values):
    print(f'Channel {channel_idx + 1}: {entropy_value}')
